## Lesson 18 - Text Analytics & Word2Vec





### Table of Contents

* [Word2vec挖掘語義相似關係](#word2vec-movie-scripts)
* [Word2vec 轉成高維空間向量 - 以字為單位](#word2vec_word)
* [Word2vec 轉成高維空間向量 - 以詞彙為單位](#word2vec_keyword)
* [計算 三詞彙 Cosine 相似度](#three_keyword)
* [Save Word2vec model](#save_model)
* [Load Word2vec model](#load_model)
* [Load text file in folder to train Word2Vec model](#Load-text-file-in-folder-to-train-Word2Vec-model)


<a id="word2vec-movie-scripts"></a>
## Word2vec挖掘語義相似關係

Word2vec is a technique for natural language processing. The word2vec algorithm uses a neural network model to learn word associations from a large corpus of text. Once trained, such a model can detect synonymous words or suggest additional words for a partial sentence. 

文章分析中也經常會用到word2vec將文字轉化為詞向量的形式，用以挖掘詞彙在語句前後的相似關係。使用word2vec測試一下模型效果。導入gensim 後，將文章轉化為模型所需的輸入形式，即可開始進行訓練，產出每個詞彙的向量格式，訓練模型的定義：

![Word2Vec（詞向量）](images/Word2Vec.png)

在Word2Vec出來之前，我們常用的主要是`one hot encoding`的方法，也就是對於每一個單字進行編碼，即用在一個位置為1，其餘位置為0的向量來表示。
向量的維度就是我們單字量的大小，向量的每一個位置，只能用來表示唯一的一個單字。

舉個例子，假設我們的有詞庫只有10個單字，分別是:
比，特，幣，昨，天，就，開，始，大，漲。 以one hot encoding的方法來表示每一個詞，那麼有下面的結果:


比 → [1,0,0,0,0,0,0,0,0,0] <br>
特 → [0,1,0,0,0,0,0,0,0,0] <br> 
幣 → [0,0,1,0,0,0,0,0,0,0] <br>
昨 → [0,0,0,1,0,0,0,0,0,0] <br>
天 → [0,0,0,0,1,0,0,0,0,0] <br>
就 → [0,0,0,0,0,1,0,0,0,0] <br>
開 → [0,0,0,0,0,0,1,0,0,0] <br>
始 → [0,0,0,0,0,0,0,1,0,0] <br>
大 → [0,0,0,0,0,0,0,0,1,0] <br>
漲 → [0,0,0,0,0,0,0,0,0,1] <br>


可以看到對於每一個單字，唯一的一個向量對它進行了表示。那麼很顯然這種表示方法至少有下面的一些缺陷

1. 在常見的距離下面，單字與單字間的距離都是沒有差別的，"比"和"特"的距離和"比"和"漲"的距離是一樣的
2. 隨著單字量的增加，向量的維度也隨之增加，對於詞庫中沒有的新字，無法以唯一的向量一對一的mapping對映
3. 當單字量較多時，高維度向量對運算速度大大降低效率，無法適應於即時的訓練與作出調整

所以，字向量的提出目的就是解決上面提到的這些問題。而英文可能是用於Word(字)，中文則是要以詞彙(至少兩個字)的向量來做訓練。

1. 如果詞彙向量有N個，那麼可以用一個n維的向量來表示每一個詞彙，並且 n 遠遠小於N，常見的n為100到300維
2. 詞彙向量每一個位置不再是只能取0和1的數值，而是可以取任意的實數
3. 詞彙向量之間的差在一定程度上是有意義的，比如，台北的詞向量為 v1，台南的詞向量為 v2, 西雅圖的詞向量為 v3 , 紐約的詞向量為 v4 ，藉由word2vec學習出來的這些詞向量大致都有相符的特徵。這個近似關係，或是說城市關係，也正是受到word2vec的啟發，在知識圖譜表示學習中，衍生了一些名為Trans的編碼演算法
4. 除此之外，地名和地名在詞向量空間中的距離比地名和動物的詞向量距離近，等等，換句話說就是描述同一屬性和種類的詞向量的距離要小於不同屬性和種類的詞向量的距離

<a id="word2vec_word"></a>
## Word2vec 轉成高維空間向量 - 以字為單位

In [1]:
from gensim.models import Word2Vec

sentences_raw = '47歲的國民黨榮譽主席連戰二公子連勝武，2009年與相戀4年的華航空姐路永佳結婚，育有1子2女，夫妻倆人前恩愛，堪稱名人界的模範夫妻，但22日《蘋果新聞網》直擊連勝武爆出婚外情，對象是身高約170公分的骨感妹！當天她與藝人朋友聚餐，連勝武獨自搭車去找她，小倆口十指緊扣步出餐廳，連勝武摟著她的腰上車轉赴酒吧續攤，酒酣耳熱之際，兩人深情撫摸著對方臉龐擁吻，喝茫的骨感妹不知傾吐什麼，哭成淚人兒，連勝武露出疼惜之情拿面紙幫她拭淚；凌晨2點多，不勝酒力的骨感妹到洗手間嘔吐，之後連勝武扶著她上車，一同返回她在中山區的住處過夜。路永佳今接受《蘋果新聞網》獨家專訪表示，「不知道這個（兩人關係）程度是到什麼程度」，她冷靜回應：「我必須跟他一起面對 ，今天如果是他做了什麼事情，他要自己出來負責，可是因為家是我們的，所以家裡的部分我要面對，我也不能逃避。」原先，路永佳答應《蘋果》，9點會請連勝武聯繫記者，但至截稿前尚未取得連勝武回應。 連勝武十指緊扣骨感妹走出餐廳。 上周五（22日）晚間8點多，記者目擊年約35歲的骨感妹，一身黑色系打扮，獨自前往內湖的高檔養生藥膳餐廳「福慧根湯」與賴薇如、熊熊等藝人聚會，烏黑長髮披肩、身高約170公分的骨感妹。用餐1小時後，骨感妹一臉開心地走出包廂到門口「煲電話粥」，言談時一臉幸福的模樣。晚間10點半，連勝武戴著口罩搭車到餐廳找她，深夜11點，連勝武與背著Chanel包的骨感妹邊戴起口罩邊步出餐廳，冷不防地隔著口罩親她的臉頰調情，再順勢牽起她的手，十指緊扣走到馬路上，還貼心幫她穿好外套，摟著她的腰等計程車。骨感妹走出餐廳外講電話，臉上流露幸福模樣。小倆口在計程車內頭靠在一起，不時隔著口罩接吻，隨後來到通化街巷內的酒吧續攤，互動宛如熱戀中的情人。小倆口點好酒開始暢聊談心，1個多小時後，兩人逐漸喝開，動作也開始放開，無法自已的牽起手來，並放到大腿上；接著她從頭開始向下撫摸至他的臉龐，順勢摸著嘴唇調情，深情地彼此凝視。接著連勝武向服務生說「再一杯」，兩人續杯暢飲，依然十指緊扣，並磨起鼻子，然後親吻臉頰，再聊了一下便熱烈地接吻起來。之後喝茫的骨感妹邊聊邊掉下眼淚，連勝武見狀抿嘴流露出不捨的神情，並拿起面紙幫她拭淚，她被安慰後反而更激動地說著，連勝武見狀立刻又拿面紙替她拭淚，並輕撫她的臉。連勝武前來餐廳帶骨感妹離開，2人十指緊扣如膠似漆。凌晨2點多，不勝酒力的骨感妹突然起身飛奔至洗手間狂吐，廁所一片狼藉滿地穢物，不知情的連勝武坐在座位上滑手機回訊息，她回座之後，小倆口繼續牽著手談心。不到10分鐘，她再度起身，此時連勝武才驚覺有異，問她：「妳還好嗎？」骨感妹二度到女廁嘔吐，頻頻發出乾嘔的聲音，連勝武起身埋單，她返回後，兩人又聊了一會兒才離開酒吧。連勝武與骨感妹上了計程車後戴上口罩，只見她將頭靠在連勝武的肩上，他也沉醉在她的溫柔鄉，看似自然地在車內隔著口罩親吻彼此。酒意未退的2人持續在車內溫存，她撫摸著連勝武的臉龐，連勝武索性將口罩脫下，忘情地激吻一陣，才又將口罩戴上。2人在女方位於中山區的住處附近下車，連勝武隨即從她身後環抱，扶著她的側胸一路走進巷內，就連在公寓門前找鑰匙時仍緊抱著。骨感妹開門後，連勝武也跟著一起入內。沒多久一輛計程車駛來，等了20幾分鐘沒人下樓便開走，直到23日清晨4點左右都未見連勝武從該公寓正門離開。對於連勝武外面行為，路永佳表示，事前沒有感受先生的生活有狀況，「但確實因疫情關係，大家都沒辦法出國，朋友都喜歡連勝武，他也愛熱鬧，所以飯局變多了，交際也變得越來越複雜，而我忙小孩，也不太干涉他，所以不知道發生什麼事情。」即便看到、聽到先生與女生過從甚密的訊息，路永佳的語氣冷靜、態度包容展現高EQ：「如果是他做了什麼事情，我必須跟他一起面對，他要出來自己負責，可是因為家是我們的，所以家裡的部分我要面對，我也不能逃避。我不知道這個（兩人關係）程度是到什麼程度，現在講離婚太突然了，我沒有辦法說，畢竟我們還有三個小孩，先看看狀況，以三個小孩為重」最後她還是為先生緩頰，「他的人緣很好，朋友都喜歡他，他也愛熱鬧。他是一個很好的人，對我也很好，他是一個很好的先生，認識他和我的朋友都知道的；至於當爸爸的部分，需要多努力一下，但是他對小孩也不錯啦。」連勝武摟著骨感妹的小蠻腰在路口等計程車。連勝武與骨感妹在計程車內耳鬢廝磨，不時隔著口罩接吻。連勝武與骨感妹到通化街巷內酒吧續攤。骨感妹從連勝武的額頭輕撫到他的嘴唇。連勝武與骨感妹情到深處，無法自己的熱吻起來。黃湯下肚，骨感妹大吐苦水邊講邊哭，連勝武面露不捨替她拭淚。連勝武埋單後，與骨感妹離開酒吧。連勝武扶著骨感妹一同搭上計程車。骨感妹（後）靠在連勝武肩上，輕撫他臉龐。連勝武摟著骨感妹在中山區下車。連勝武攙扶著骨感妹，一同走向張女住處。連勝武（前）與骨感妹一起走進女方家門，直至清晨未見他離去。連勝武和路永佳結婚11年。連勝武和路永佳結婚11年。'
model = Word2Vec(sentences_raw, sg=1, size=250,  window=5,  min_count=1,  negative=3, sample=0.001, hs=1, workers=4)  

In [2]:
len(model.wv.index2word)

522

In [3]:
# 檢視字詞
for i in range(9,20):
    print(model.wv.index2word[i])

妹
著
一
他
人
不
到
情
車
口
後


In [4]:
vector = model['妹']
vector

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-1.7949782e-03, -1.2461718e-03, -1.5766241e-03,  1.9535627e-03,
        1.7471313e-03, -1.8572459e-03,  1.5724741e-03, -1.8176987e-03,
       -3.0904743e-04,  1.8574044e-04, -1.6251500e-03, -1.4588892e-03,
       -8.1168761e-04, -1.7675897e-03,  1.1630853e-03, -1.7692036e-03,
        7.4888836e-04,  4.1992177e-04, -1.7930850e-04,  1.4886686e-03,
       -1.4293054e-03, -1.8114827e-03,  1.3400569e-03, -9.8771381e-04,
        5.7287520e-04,  1.8858287e-03,  6.6864042e-04,  6.5233104e-04,
       -1.8636630e-03,  5.9006538e-04,  2.0712326e-04, -4.0441073e-04,
        1.6341800e-03, -1.7758164e-03,  2.0429754e-04,  1.7850052e-03,
        7.7052385e-04, -1.5735216e-03,  5.1568745e-04, -1.3533053e-03,
       -1.7080347e-03, -1.6521226e-03,  1.4789554e-03,  7.2403607e-04,
        1.3599575e-03,  1.1410867e-03, -1.4108787e-03, -1.1883867e-03,
       -1.8731265e-03,  7.4646570e-04, -5.0212367e-04, -6.5935514e-04,
        1.6013201e-03, -1.4073316e-03, -1.4279857e-03, -1.2483587e-03,
      

In [14]:
# 顯示空間和字距離相近的字
res = model.most_similar('妹', topn=100)  # 此字有出現在 corpus 當中
for item in res[:5]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

藥, 0.17
直, 0.17
唇, 0.16
水, 0.16
麼, 0.15


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [15]:
# word '婚外情' not in vocabulary
vector = model['婚外情']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


KeyError: "word '婚外情' not in vocabulary"

In [16]:
# 計算兩字的相似度
model.similarity('妹', '吻')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.0103410585

<a id="word2vec_keyword"></a>
## Word2vec 轉成高維空間向量 - 以詞彙為單位

In [17]:
import jieba

jieba.initialize()
jieba.load_userdict('dict/mydic.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\princ\AppData\Local\Temp\jieba.cache
Loading model cost 0.708 seconds.
Prefix dict has been built succesfully.


In [18]:
def jiebaCut(x):
    punct = set(u''' :!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
    # punct |= set(exception)
    words = [word.strip().replace('\n','') for word in jieba.cut(x, cut_all=False) if (len(word.strip()) >= 2) and ( not any(ext in word for ext in punct) )]
    return words

sentences_list = []
sentences_list.append(sentences_raw)
sentences_list.append('日前蔡依珊出席時尚活動，提及小叔連勝武約會骨感妹被拍到，近期是否成為家人聊天之間關心的話題？蔡依珊正色表示，「因為我不是當事人，真的不便做評論。」接著被問到自己有沒有什麼經營婚姻之道，可以避免野花的出現，她表示，「不敢說自己有什麽婚姻經營之道，就是把自己顧好，夫妻互相尊重。」關於把自己顧好的部分，也不是說要維持在最佳巔峰狀態，「那樣真的太累了，其實就是吃好、睡好，也不用給自己太大壓力，看心情吧，想放鬆就放鬆，想打扮的時候也可以好好的打理一番，不用說很刻意。」')
sentences_list.append('連家小兒子連勝武爆出婚外情，哥哥連勝文今天(1/30)到內湖參加單車公益活動時，不斷被媒體追問，對於弟弟外遇的事情，連勝文：「(家人知道嗎？) 呃...應該會知道吧。我沒有聽說(父親知道)什麼東西，因為我只有跟我弟弟說，希望他未來能夠謹言慎行。」')

sentences = list(map(jiebaCut, sentences_list))
model = Word2Vec(sentences, size=250,  window=5,  min_count=1,  negative=3, workers=4)

In [19]:
len(model.wv.index2word)

449

In [20]:
#文字以向量形式呈現
model.wv.vectors

array([[-1.9739293e-03, -1.6404660e-03,  1.3326459e-03, ...,
        -1.7136821e-05, -2.0090824e-03,  1.8703668e-04],
       [-1.5786896e-03,  9.0298452e-04, -1.1577907e-03, ...,
         1.1557548e-03,  9.9379336e-04, -1.7935510e-03],
       [-6.6615292e-04,  1.0790558e-03,  4.2394520e-04, ...,
        -9.9070638e-04, -3.8672163e-04, -1.1404554e-03],
       ...,
       [ 1.7132302e-03, -2.9933444e-04, -1.9530570e-03, ...,
        -1.4407132e-03, -1.2370271e-03,  7.8031677e-04],
       [ 1.7427499e-03, -4.7618491e-04,  8.1199854e-05, ...,
        -7.5259717e-04, -1.0728630e-04,  1.7445704e-03],
       [-6.0563965e-04,  5.5704114e-04,  1.2025083e-03, ...,
         8.8062504e-04,  1.6316262e-03,  1.5058012e-03]], dtype=float32)

In [21]:
for x in model.wv.index2word[:10]:
    print(x)

連勝武
骨感妹
口罩
自己
路永佳
餐廳
知道
什麼
十指
緊扣


In [22]:
# 顯示空間和字距離相近的詞彙
res = model.most_similar('骨感妹', topn=100)  # 此詞彙有出現在 corpus 當中
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

談心, 0.17
深情, 0.16
起來, 0.15
左右, 0.15
牽著手, 0.14
點半, 0.14
先生, 0.13
眼淚, 0.13
前來, 0.12
路上, 0.12


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [23]:
# 計算 兩詞彙 Cosine 相似度
res = model.similarity('連勝武', '骨感妹')
print(res)

-0.07379925


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [24]:
res = model.similarity('連勝武', '路永佳')
print(res)

-0.024684161


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [28]:
# 顯示空間和字距離相近的詞彙
res = model.most_similar('連勝武', topn=100)  # 此詞彙有出現在 corpus 當中
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

不用, 0.24
深情, 0.18
為重, 0.17
所以, 0.16
返回, 0.14
內耳鬢, 0.14
十指, 0.13
還貼, 0.13
只見, 0.13
小時, 0.13


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [40]:
# 計算 兩詞彙 Cosine 相似度
res = model.similarity('連勝武', '十指')
print(res)

0.12965415


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [50]:
res = model.most_similar('婚外情', topn=100)  # 此詞彙有出現在 corpus 當中
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

溫柔, 0.18
深處, 0.17
邊講邊, 0.17
緩頰, 0.17
附近, 0.15
過從, 0.14
疫情, 0.14
關心, 0.14
好好, 0.14
展現, 0.14


### 重新校稿一次，這次將他與她替換為人名
「連勝武」與「骨感妹」空間距離為負，反倒是與妻子路永佳距離為正，回顧新聞描述，大多以人名加上第三人稱組成一句，所以懷疑是否是Word2vec學不會主動詞，所以再手動將文章中的他與她置換為人名，重新訓練。

In [33]:
sentences_revised = '47歲的國民黨榮譽主席連戰二公子連勝武，2009年與相戀4年的華航空姐路永佳結婚，育有1子2女，夫妻倆人前恩愛，堪稱名人界的模範夫妻，但22日《蘋果新聞網》直擊連勝武爆出婚外情，連勝武外遇對象是身高約170公分的骨感妹！當天骨感妹與藝人朋友聚餐，連勝武獨自搭車去找骨感妹，小倆口十指緊扣步出餐廳，連勝武摟著骨感妹的腰上車轉赴酒吧續攤，酒酣耳熱之際，兩人深情撫摸著對方臉龐擁吻，喝茫的骨感妹不知傾吐什麼，哭成淚人兒，連勝武露出疼惜之情拿面紙幫骨感妹拭淚；凌晨2點多，不勝酒力的骨感妹到洗手間嘔吐，之後連勝武扶著骨感妹上車，一同返回骨感妹在中山區的住處過夜。路永佳今接受蘋果新聞網獨家專訪表示，「不知道兩人關係程度是到什麼程度」，路永佳冷靜回應：「路永佳必須跟連勝武一起面對 ，今天如果是他做了什麼事情，連勝武要自己出來負責，可是因為家是我們的，所以家裡的部分我要面對，我也不能逃避。」原先，路永佳答應蘋果，9點會請連勝武聯繫記者，但至截稿前尚未取得連勝武回應。 連勝武十指緊扣骨感妹走出餐廳。 上周五（22日）晚間8點多，記者目擊年約35歲的骨感妹，一身黑色系打扮，獨自前往內湖的高檔養生藥膳餐廳「福慧根湯」與賴薇如、熊熊等藝人聚會，烏黑長髮披肩、身高約170公分的骨感妹。用餐1小時後，骨感妹一臉開心地走出包廂到門口「煲電話粥」，言談時一臉幸福的模樣。晚間10點半，連勝武戴著口罩搭車到餐廳找骨感妹，深夜11點，連勝武與背著Chanel包的骨感妹邊戴起口罩邊步出餐廳，連勝武冷不防地隔著口罩親骨感妹的臉頰調情，連勝武再順勢牽起骨感妹的手，十指緊扣走到馬路上，連勝武還貼心幫骨感妹穿好外套，連勝武摟著骨感妹的腰等計程車。骨感妹走出餐廳外講電話，臉上流露幸福模樣。小倆口在計程車內頭靠在一起，不時隔著口罩接吻，隨後來到通化街巷內的酒吧續攤，互動宛如熱戀中的情人。小倆口點好酒開始暢聊談心，1個多小時後，兩人逐漸喝開，動作也開始放開，無法自已的牽起手來，並放到大腿上；接著她從頭開始向下撫摸至他的臉龐，順勢摸著嘴唇調情，連勝武與骨感妹深情地彼此凝視。接著連勝武向服務生說「再一杯」，兩人續杯暢飲，依然十指緊扣，並磨起鼻子，然後連勝武親吻骨感妹的臉頰，再聊了一下便熱烈地接吻起來。之後喝茫的骨感妹邊聊邊掉下眼淚，連勝武見狀抿嘴流露出不捨的神情，連勝武拿起面紙幫骨感妹拭淚，骨感妹被安慰後反而更激動地說著，連勝武見狀立刻又拿面紙替骨感妹拭淚，連勝武輕撫骨感妹的臉。連勝武前來餐廳帶骨感妹離開，2人十指緊扣如膠似漆。凌晨2點多，不勝酒力的骨感妹突然起身飛奔至洗手間狂吐，廁所一片狼藉滿地穢物，不知情的連勝武坐在座位上滑手機回訊息，骨感妹回座之後，小倆口繼續牽著手談心。不到10分鐘，骨感妹再度起身，此時連勝武才驚覺有異，問骨感妹：「妳還好嗎？」骨感妹二度到女廁嘔吐，頻頻發出乾嘔的聲音，連勝武起身埋單，骨感妹返回後，兩人又聊了一會兒才離開酒吧。連勝武與骨感妹上了計程車後戴上口罩，只見骨感妹將頭靠在連勝武的肩上，連勝武也沉醉在骨感妹的溫柔鄉，看似自然地在車內隔著口罩親吻彼此。酒意未退的2人持續在車內溫存，骨感妹撫摸著連勝武的臉龐，連勝武索性將口罩脫下，忘情地激吻一陣，才又將口罩戴上。2人在女方位於中山區的住處附近下車，連勝武隨即從骨感妹身後環抱，連勝武扶著骨感妹的側胸一路走進巷內，就連在公寓門前找鑰匙時仍緊抱著。骨感妹開門後，連勝武也跟著一起入內。沒多久一輛計程車駛來，等了20幾分鐘沒人下樓便開走，直到23日清晨4點左右都未見連勝武從該公寓正門離開。對於連勝武外面行為，路永佳表示，事前沒有感受先生的生活有狀況，「但確實因疫情關係，大家都沒辦法出國，朋友都喜歡連勝武，他也愛熱鬧，所以飯局變多了，交際也變得越來越複雜，而我忙小孩，也不太干涉他，所以不知道發生什麼事情。」即便看到、聽到先生與女生過從甚密的訊息，路永佳的語氣冷靜、態度包容展現高EQ：「如果是他做了什麼事情，我必須跟他一起面對，他要出來自己負責，可是因為家是我們的，所以家裡的部分我要面對，我也不能逃避。我不知道這個（兩人關係）程度是到什麼程度，現在講離婚太突然了，我沒有辦法說，畢竟我們還有三個小孩，先看看狀況，以三個小孩為重」最後她還是為先生緩頰，「他的人緣很好，朋友都喜歡他，他也愛熱鬧。他是一個很好的人，對我也很好，他是一個很好的先生，認識他和我的朋友都知道的；至於當爸爸的部分，需要多努力一下，但是他對小孩也不錯啦。」連勝武摟著骨感妹的小蠻腰在路口等計程車。連勝武與骨感妹在計程車內耳鬢廝磨，不時隔著口罩接吻。連勝武與骨感妹到通化街巷內酒吧續攤。骨感妹從連勝武的額頭輕撫到他的嘴唇。連勝武與骨感妹情到深處，無法自己的熱吻起來。黃湯下肚，骨感妹大吐苦水邊講邊哭，連勝武面露不捨替她拭淚。連勝武埋單後與骨感妹離開酒吧。連勝武扶著骨感妹一同搭上計程車。骨感妹（後）靠在連勝武肩上，輕撫他臉龐。連勝武摟著骨感妹在中山區下車。連勝武攙扶著骨感妹，一同走向張女住處。連勝武（前）與骨感妹一起走進女方家門，直至清晨未見他離去。連勝武和路永佳結婚11年。連勝武和路永佳結婚11年。日前蔡依珊出席時尚活動，提及小叔連勝武約會骨感妹被拍到，近期是否成為家人聊天之間關心的話題？蔡依珊正色表示，「因為我不是當事人，真的不便做評論。」接著被問到自己有沒有什麼經營婚姻之道，可以避免野花的出現，她表示，「不敢說自己有什麽婚姻經營之道，就是把自己顧好，夫妻互相尊重。」關於把自己顧好的部分，也不是說要維持在最佳巔峰狀態，「那樣真的太累了，其實就是吃好、睡好，也不用給自己太大壓力，看心情吧，想放鬆就放鬆，想打扮的時候也可以好好的打理一番，不用說很刻意。」連家小兒子連勝武爆出婚外情，哥哥連勝文今天(1/30)到內湖參加單車公益活動時，不斷被媒體追問，對於弟弟外遇的事情，連勝文：「(家人知道嗎？) 呃...應該會知道吧。我沒有聽說(父親知道)什麼東西，因為我只有跟我弟弟說，希望他未來能夠謹言慎行。」'
sentences_revised_list = sentences_revised.split('，')
sentences = list(map(jiebaCut, sentences_revised_list))

In [34]:
sentences[:4]

[['47', '國民黨', '榮譽', '主席', '連戰', '公子', '連勝武'],
 ['2009', '相戀', '航空', '路永佳', '結婚'],
 ['育有'],
 ['夫妻', '前恩愛']]

In [35]:
model = Word2Vec(sentences, size=250,  window=5,  min_count=1,  negative=3, workers=4)
for x in model.wv.index2word[:10]:
    print(x)

連勝武
骨感妹
路永佳
口罩
自己
餐廳
知道
什麼
十指
緊扣


In [36]:
# 計算 兩詞彙 Cosine 相似度
res = model.similarity('連勝武', '骨感妹')
print(res)

-0.07319609


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [38]:
# 計算 兩詞彙 Cosine 相似度
res = model.similarity('連勝武', '親吻')
print(res)

-0.03360855


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


### Conclusion

得到幾乎相同的模型，即使在同一句話上下文都提到，也不會因此改變空間上的距離，這是因為每一句話描述的都是`不同的事`，所以不能找到描述A時，模型推估這個位置應該要填上B所致，藉此訓練，讓我們更理解Word2vec的模型結果。

<a id="three_keyword"></a>
## 計算 三詞彙 Cosine 相似度

In [44]:
res = model.most_similar(['連勝武', '骨感妹'], ['親吻'], topn= 100)
print("%s之於%s，如%s之於: " % ('連勝武', '骨感妹', '親吻'))
for item in res[:10]:
    print("{}, {}".format(item[0], str(item[1])))

連勝武之於骨感妹，如親吻之於: 
深情, 0.24562428891658783
點半, 0.21018190681934357
只見, 0.1992698460817337
20, 0.13939644396305084
尚未, 0.13560235500335693
入內, 0.1355920135974884
嘔吐, 0.13272947072982788
先生, 0.13181033730506897
街巷, 0.13078582286834717
直擊, 0.13024142384529114


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


<a id="save_model"></a>
## Save Word2vec model

In [45]:
modelname = "model/Word2Vec/word2vec_news.bin"
model.wv.save_word2vec_format(modelname, binary=True)

<a id="load_model"></a>
## Load Word2vec model

In [46]:
from gensim.models import word2vec, keyedvectors

modelname = "model/Word2Vec/word2vec_news.bin"
model = keyedvectors.KeyedVectors.load_word2vec_format(modelname,binary=True)

In [51]:
# 測試 Word2Vec model
res = model.similarity('骨感妹', '談心')
print(res)

0.17089297


<a id="Load-text-file-in-folder-to-train-Word2Vec-model"></a>
## Load text file in folder to train Word2Vec model

In [52]:
import os
import jieba
from gensim.models import Word2Vec

jieba.initialize()
jieba.load_userdict('dict/mydic.txt')

class Sentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
        jieba.load_userdict("dict/mydic.txt")

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                punct = set(u''' :!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
                # punct |= set(exception)
                words = [word for word in jieba.cut(line, cut_all=False) if (len(word.strip()) >= 2) and ( not any(ext in word for ext in punct) )]
                yield words

In [54]:
# 新聞
folder_path = "./data/news/"
sentences = Sentences(folder_path)
model = Word2Vec(sentences, size=250,  window=5,  min_count=1,  negative=3, workers=4)

In [55]:
model.wv.index2word[:20]

['連勝武',
 '骨感妹',
 '路永佳',
 '賴薇如',
 '蘋果',
 '新聞',
 '外遇',
 '自己',
 '知道',
 '朋友',
 '表示',
 '小孩',
 '媒體',
 '一起',
 '中心',
 '連戰',
 '就是',
 'IG',
 '回應',
 '記者']

### Save corpus Word2Vec Model

In [56]:
modelname = "model/Word2Vec/word2vec_corpus.bin"
model.wv.save_word2vec_format(modelname, binary=True)

### Load corpus Word2Vec Model

In [57]:
from gensim.models import word2vec, keyedvectors

modelname = "model/Word2Vec/word2vec_corpus.bin"
model = keyedvectors.KeyedVectors.load_word2vec_format(modelname,binary=True)

In [58]:
model.wv.index2word

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


['連勝武',
 '骨感妹',
 '路永佳',
 '賴薇如',
 '蘋果',
 '新聞',
 '外遇',
 '自己',
 '知道',
 '朋友',
 '表示',
 '小孩',
 '媒體',
 '一起',
 '中心',
 '連戰',
 '就是',
 'IG',
 '回應',
 '記者',
 '獨家',
 'APP',
 '看到',
 '連勝文',
 '掌握',
 '婚姻',
 '一個',
 '大家',
 '免費',
 '婚外情',
 '不是',
 '報導',
 '無廣告',
 '爆料',
 '特勤',
 '國民黨',
 '11',
 '新聞網',
 '什麼',
 '網友',
 '女兒',
 '夫妻',
 '娛樂',
 '富商',
 '我們',
 '下載',
 '真的',
 '爆出',
 '所以',
 '熊熊',
 '閨密',
 '主席',
 '因為',
 '女方',
 '時事',
 '妻子',
 '沒有',
 '孩子',
 '結婚',
 '餐廳',
 '升級',
 '壹會員',
 '獨享',
 '事情',
 '可以',
 '完整',
 '兩人',
 '豪門',
 '不能',
 '還是',
 '28',
 '覺得',
 '處理',
 '好友',
 '甚至',
 '提供',
 '照片',
 '翻攝',
 '小模',
 '榮譽',
 '富豪',
 '友人',
 '這個',
 '可能',
 '即點',
 '臉書',
 '只有',
 '不過',
 '先生',
 '還有',
 '透露',
 '閨蜜',
 '現在',
 '過夜',
 '老婆',
 '一名',
 '老公',
 '認識',
 '台灣',
 '清楚',
 '仔直擊',
 '偷吃',
 '家庭',
 '大人',
 '只是',
 '酒吧',
 '直擊',
 '口罩',
 '認為',
 '怎麼',
 '發現',
 '他們',
 '連家',
 '這樣',
 '正宮',
 '起底',
 '而且',
 '女星',
 '面對',
 '台北',
 '子女',
 '曝光',
 '如果',
 '小三',
 '伴遊',
 '生活',
 '10',
 '弟弟',
 '疫苗',
 '女生',
 '次子',
 '一位',
 '姓小模',
 '冰箱',
 '事件',
 '日前',
 '親密',
 '男人',
 '開始',
 '多年',
 '歐洲',


In [59]:
# 測試訓練好的詞向量模型，使用 model[keyWord] 取出 keyword 這個詞彙的詞向量
keyWord = '外遇'
print(model[keyWord])

[-2.96347914e-03 -3.30780429e-04  6.43712818e-04 -7.23797304e-04
  2.01448472e-03 -2.59222812e-03 -3.24690569e-04  5.67208102e-04
  8.06614105e-03  1.09496329e-03  4.93030879e-04  2.06478941e-03
  2.75848812e-04 -8.80425912e-04 -2.23653973e-03  1.29257515e-03
 -4.77554349e-05  2.63878290e-04 -3.66014452e-03  9.08842660e-04
  4.37977770e-03 -6.75586169e-04 -3.06305895e-03  3.08475527e-03
  3.22636543e-03 -1.49841735e-03  1.32640079e-03  1.01776570e-02
  2.35088286e-03  5.29658901e-05  6.21119421e-03  6.50613662e-03
  4.22141794e-03 -4.99548996e-03  7.53667700e-05  1.69475900e-03
  2.31890334e-03 -2.65312288e-03 -1.68244576e-03  2.72793142e-04
  3.97135271e-03  3.75816529e-03  3.65428324e-03  2.25795386e-03
  7.40123587e-03  7.74662057e-03  4.40187380e-03  2.52083549e-03
 -4.43423633e-03  2.24489602e-04 -4.07959521e-03  2.53182533e-03
 -3.49686574e-03  2.86570750e-03 -2.75145238e-03  8.51622783e-04
  6.65672764e-04 -1.35641615e-03  2.34290143e-03  2.32370640e-03
  4.42383811e-03 -7.07902

In [60]:
# 顯示空間和字距離相近的關聯詞彙
res = model.most_similar('婚外情',topn = 100)  # 此詞彙有出現在 corpus 當中
for item in res[:10]:
    print("{}, {}".format(item[0], str(round(item[1], 2))) )

知道, 0.81
爆料, 0.81
媒體, 0.8
自己, 0.8
照片, 0.79
回應, 0.79
夫妻, 0.79
爆出, 0.78
朋友, 0.78
11, 0.78


In [61]:
model.similarity('婚外情', '爆料')

0.80834943

## Summary

如何評價所建「詞向量」的好壞？

依照 word2vec 的原理，詞意相近的詞在向量空間當中的距離是接近的，但會因為「文本內容性質的差異」而有所不同，例如在新聞類的文本中，「台灣」字詞會常與地名或時事事件等字詞距離接近；然而若是跟 PTT 論壇相關的文本，「台灣」字詞可能會更常與「鄉民/島民/魯蛇/溫拿」這類聊天用語的字詞距離靠近；總之，可以透過所訓練之詞向量距離，觀察他們語意是否相近去做衡量。

其次是準備訓練集，必須將文章整理成一列(row)一篇，組成的語料(corpus)若太少，則無法歸納出較好的相似性，因為機器學習，是要讓機器從語料中找到規則，所以大量的語料是進行斷詞與Word2vec必要條件之一。